# Building an Anime Recommender System

In [1]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 1. Load Dataset and Model

In [2]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 3. Webscrape current profile and create prediction

In [3]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 167 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7


In [4]:
rhat = model.predict(x)

In [5]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [6]:
prediction.sort_values(by='score', ascending=False).head(20)

,score,title,title_english,type,premiered,genre
11981,10.144850,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,Puella Magi Madoka Magica the Movie: Rebellion,Movie,NaN,"Psychological, Drama, Magic, Thriller"
11757,10.105658,Sword Art Online,Sword Art Online,TV,Summer 2012,"Action, Adventure, Fantasy, Game, Romance"
13125,9.843902,Shinsekai yori,From the New World,TV,Fall 2012,"Sci-Fi, Mystery, Horror, Psychological, Supern..."
21881,9.404726,Sword Art Online II,Sword Art Online II,TV,Summer 2014,"Action, Game, Adventure, Romance, Fantasy"
11979,9.384899,Mahou Shoujo Madoka★Magica Movie 2: Eien no Mo...,Puella Magi Madoka Magica the Movie Part 2: Et...,Movie,NaN,"Drama, Magic, Psychological, Thriller"
11977,9.198149,Mahou Shoujo Madoka★Magica Movie 1: Hajimari n...,Puella Magi Madoka Magica the Movie Part 1: Be...,Movie,NaN,"Psychological, Drama, Magic, Thriller"
3785,9.118061,Evangelion: 3.0 You Can (Not) Redo,Evangelion: 3.0 You Can (Not) Redo,Movie,NaN,"Action, Sci-Fi, Mecha"
23281,9.007453,Psycho-Pass 2,Psycho-Pass 2,TV,Fall 2014,"Action, Sci-Fi, Police, Psychological"
5,8.793917,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,NaN,"Action, Space, Drama, Mystery, Sci-Fi"
4896,8.744678,Umineko no Naku Koro ni,Umineko: When They Cry,TV,Summer 2009,"Mystery, Horror, Psychological, Supernatural"


In [7]:
users = pandas.read_csv("../data/raw/UserList.csv")
idx = users.index[users.username == 'Manuel'][0]

In [8]:
rhat = model.U[idx].dot(model.V.T) + model.bu[idx] + model.bv + model.mu

In [9]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [10]:
prediction.sort_values(by='score', ascending=False).head(20)

,score,title,title_english,type,premiered,genre
32995,10.715361,Yuri!!! on Ice,Yuri!!! On ICE,TV,Fall 2016,"Comedy, Sports"
10161,10.666838,No.6,No. 6,TV,Summer 2011,"Action, Sci-Fi, Mystery, Drama"
33352,10.570225,Violet Evergarden,Violet Evergarden,TV,Winter 2018,"Slice of Life, Drama, Fantasy"
35062,10.508804,Mahoutsukai no Yome,The Ancient Magus&#039; Bride,TV,Fall 2017,"Slice of Life, Fantasy, Magic, Shounen"
34240,10.330217,Shelter,Shelter,Music,NaN,"Sci-Fi, Music"
10321,10.065937,Uta no☆Prince-sama♪ Maji Love 1000%,Uta no Prince Sama,TV,Summer 2011,"Harem, Music, Comedy, Romance, School, Shoujo"
431,10.022015,Howl no Ugoku Shiro,Howl&#039;s Moving Castle,Movie,NaN,"Adventure, Drama, Fantasy, Romance"
32729,10.007061,Orange,Orange,TV,Summer 2016,"Sci-Fi, Drama, Romance, School, Shoujo"
22265,9.996372,Free!: Eternal Summer,Free! - Eternal Summer,TV,Summer 2014,"Slice of Life, Comedy, Sports, Drama, School"
731,9.979772,Interstella5555: The 5tory of The 5ecret 5tar ...,NaN,Music,NaN,"Adventure, Drama, Music, Sci-Fi"


In [7]:
print("Hello Isabel!")

Hello Isabel!
